# Göstergelerin Yazılması

Bu not defterinde **PyBroker**'da özel hisse senedi göstergelerinin nasıl oluşturulacağı ve entegre edileceği açıklanmaktadır. **PyBroker**'daki göstergeler, sayısal hesaplamaya yönelik güçlü bir kitaplık olan [NumPy](https://numpy.org/) kullanılarak yazılmıştır. Performansı optimize etmek için Python kodunu verimli makine koduna çeviren bir JIT derleyicisi olan [Numba](https://numba.pydata.org/)'dan da yararlanacağız. Numba özellikle döngüleri ve NumPy dizilerini içeren kodu hızlandırmak için faydalıdır. Bu kütüphaneleri şu şekilde içe aktarıyoruz:

In [1]:
import numpy as np
from numba import njit

Aşağıdaki kod, kapanış fiyatlarını hareketli ortalama (CMMA) çıkararak hesaplayan ve [ortalamanın tersine çevrilmesi](https://en.wikipedia.org/wiki/Mean_reversion_(finance)) stratejisi için kullanılabilecek bir gösterge işlevini gösterir:

In [2]:
def cmma(bar_data, lookback):

    @njit  # Numba JIT'i etkinleştirin.
    def vec_cmma(values):
        # Sonuç dizisini başlatın.
        n = len(values)
        out = np.array([np.nan for _ in range(n)])
        
        # Yeniden incelemeden başlayan tüm çubuklar için:
        for i in range(lookback, n):
            # Yeniden incelemenin hareketli ortalamasını hesaplayın.
            ma = 0
            for j in range(i - lookback, i):
                ma += values[j]
            ma /= lookback
            # Hareketli ortalamayı değerden çıkarın.
            out[i] = values[i] - ma
        return out
    
    # Yakın fiyatlarla hesaplayın.
    return vec_cmma(bar_data.close)

``cmma`` işlevi iki bağımsız değişken alır: [BarData](https://www.pybroker.com/en/latest/reference/pybroker.common) örneği olan ```bar_data```. OHLCV verilerini ve özel alanları tutan .html#pybroker.common.BarData) sınıfını ve hareketli ortalamanın yeniden incelemesi için kullanıcı tanımlı bir bağımsız değişken olan "geri inceleme"yi içerir.

```vec_cmma``` işlevi Numba tarafından JIT derlenmiştir ve ```cmma``` nın içine yerleştirilmiştir. Numba derlenmiş işlevi argüman olarak NumPy dizisini desteklediğinden ancak ```BarData``` gibi bir Python sınıfının örneğini desteklemediğinden bu gereklidir. Gösterge değerlerinin hesaplanmasının Numba tarafından [vektörleştirildiğini](https://en.wikipedia.org/wiki/Array_programming) unutmayın; bu, tüm geçmiş veriler üzerinde aynı anda gerçekleştirildiği anlamına gelir. Bu yaklaşım geriye dönük test sürecini önemli ölçüde hızlandırır.

Bir sonraki adım, aşağıdaki kodu kullanarak gösterge işlevini **PyBroker**'a kaydetmektir:

In [3]:
import pybroker

cmma_20 = pybroker.indicator('cmma_20', cmma, lookback=20)

Burada gösterge fonksiyonuna ``cmma_20`` adını veriyoruz ve ``lookback`` parametresini ```20``` bar olarak belirtiyoruz. Gösterge işlevinde ```bar_data```dan sonra gelen tüm bağımsız değişkenler, kullanıcı tanımlı bağımsız değişkenler olarak [pybroker.indicator](https://www.pybroker.com/en/latest/reference/pybroker.indicator) öğesine iletilecektir. .html#pybroker.indicator.indicator). Gösterge işlevi **PyBroker**'e kaydolduktan sonra yeni bir [Gösterge](https://www.pybroker.com/en/latest/reference/pybroker.indicator.html#pybroker.indicator.Indicator) döndürecektir. tanımladığımız gösterge fonksiyonuna başvuran örnek.

Aşağıda **PyBroker**'da kayıtlı ```Gösterge```nin [Yahoo Finance](https://finance.yahoo.com)'dan indirilen bazı verilerle nasıl kullanılacağına dair bir örnek verilmiştir:

In [4]:
from pybroker import YFinance

pybroker.enable_data_source_cache('yfinance')

yfinance = YFinance()
df = yfinance.query('PG', '4/1/2020', '4/1/2022')

Loading bar data...


[*********************100%***********************]  1 of 1 completed
Loaded bar data: 0:00:00 



In [5]:
cmma_20(df)

2020-04-01         NaN
2020-04-02         NaN
2020-04-03         NaN
2020-04-06         NaN
2020-04-07         NaN
                ...   
2022-03-25    1.967502
2022-03-28    3.288005
2022-03-29    4.968507
2022-03-30    3.790999
2022-03-31    2.171002
Length: 505, dtype: float64

Gördüğünüz gibi ```Indicator``` örneği bir ``Callable``dır. Çağrıldıktan sonra, elde edilen hesaplanan gösterge değerleri bir [Pandas Serisi](https://pandas.pydata.org/docs/reference/api/pandas.Series.html) olarak döndürülür.

```Indicator``` sınıfı ayrıca bilgi içeriğini ölçmek için işlevler sağlar. Örneğin, [çeyrekler arası aralığı (IQR)](https://en.wikipedia.org/wiki/Interquartile_range) hesaplayabilirsiniz:

In [6]:
cmma_20.iqr(df)

4.655495452880842

Veya göreceli [entropiyi](https://en.wikipedia.org/wiki/Entropy_(information_theory)) hesaplayın:

In [7]:
cmma_20.relative_entropy(df)

0.7495800114455111

## Göstergeyi Bir Stratejide Kullanmak

Göstergemizi uyguladıktan sonraki adım, onu bir ticaret stratejisine entegre etmektir. Aşağıdaki örnek, 20 günlük CMMA 0'dan küçük olduğunda, yani son kapanış fiyatı 20 günlük hareketli ortalamanın altına düştüğünde uzun vadeli basit bir stratejiyi göstermektedir:

In [8]:
def buy_cmma_cross(ctx):
    if ctx.long_pos():
        return
    # 20 günlük CMMA'nın en güncel değeri < 0 ise satın alma emri verin:
    if ctx.indicator('cmma_20')[-1] < 0:
        ctx.buy_shares = ctx.calc_target_shares(1)
        ctx.hold_bars = 3

Gösterge değerleri, [ExecContext](https) üzerinde [ctx.indicator](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.BaseContext.indicator) çağrılarak alınır. ://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext) ve ```cmma_20``` göstergesinin kayıtlı adını aktararak.

(Not: Sembolü [ExecContext#indicator()](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context) öğesine ileterek başka bir sembole ilişkin gösterge verilerini de alabilirsiniz. .ExecContext.indicator))

In [9]:
from pybroker import Strategy

strategy = Strategy(yfinance, '4/1/2020', '4/1/2022')
strategy.add_execution(buy_cmma_cross, 'PG', indicators=cmma_20)

```buy_cmma_cross``` işlevi [Stratejiye](https://www.pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy) ile birlikte eklenir. ```cmma_20``` göstergesi. Hesaplanan gösterge değerlerinin diske önbelleğe alınmasını aşağıdaki yöntemlerle etkinleştirebiliriz:

In [10]:
pybroker.enable_indicator_cache('my_indicators')

Son olarak aşağıdaki kod ile backtest’i çalıştırabiliriz. "Isınma" bağımsız değişkeni, arka test yürütmesini çalıştırmadan önce 20 çubuğun geçmesi gerektiğini belirtir:

In [11]:
result = strategy.backtest(warmup=20)
result.metrics_df.round(4)

Backtesting: 2020-04-01 00:00:00 to 2022-04-01 00:00:00

Loaded cached bar data.

Computing indicators...


  0% (0 of 1) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--
100% (1 of 1) |##########################| Elapsed Time: 0:00:00 ETA:  00:00:00
100% (1 of 1) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00



Test split: 2020-04-01 00:00:00 to 2022-03-31 00:00:00


  0% (0 of 505) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
100% (505 of 505) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:02


,name,value
0,trade_count,60.0000
1,initial_market_value,100000.0000
2,end_market_value,100759.3600
3,total_pnl,759.3600
4,unrealized_pnl,0.0000
5,total_return_pct,0.7594
6,total_profit,41596.7500
7,total_loss,-40837.3900
8,total_fees,0.0000
9,max_drawdown,-13446.9300


Geri test çalıştırıldığında **PyBroker** gösterge değerlerini hesaplar. ``Strateji``ye birden fazla gösterge eklenmişse **PyBroker** bunları birden fazla CPU çekirdeğinde paralel olarak hesaplar.

## Vektörleştirilmiş Yardımcılar

**PyBroker** kitaplığı, göstergelerin hesaplanması sürecini kolaylaştırmak için vektörleştirilmiş yardımcı işlevler sağlar. Bu yardımcı işlevlerden biri, *n'nin her dönemi için en yüksek değeri hesaplayan [yüksekv](https://www.pybroker.com/en/latest/reference/pybroker.vect.html#pybroker.vect.highv) işlevidir. *Barlar.

Örnek kodda, 5 çubukluk her dönem için *en yüksek* en yüksek fiyatı hesaplamak için ```yüksek``` kullanan ```hhv``` adlı bir gösterge işlevi tanımlanmıştır:

In [12]:
from pybroker import highv

def hhv(bar_data, period):
    return highv(bar_data.high, period)

hhv_5 = pybroker.indicator('hhv_5', hhv, period=5)
hhv_5(df)

2020-04-01           NaN
2020-04-02           NaN
2020-04-03           NaN
2020-04-06           NaN
2020-04-07    120.059998
                 ...    
2022-03-25    153.919998
2022-03-28    153.919998
2022-03-29    156.470001
2022-03-30    156.470001
2022-03-31    156.470001
Length: 505, dtype: float64

In [13]:
from pybroker import highest

hhv_5 = highest('hhv_5', 'high', period=5)
hhv_5(df)

2020-04-01           NaN
2020-04-02           NaN
2020-04-03           NaN
2020-04-06           NaN
2020-04-07    120.059998
                 ...    
2022-03-25    153.919998
2022-03-28    153.919998
2022-03-29    156.470001
2022-03-30    156.470001
2022-03-31    156.470001
Length: 505, dtype: float64

## Çoklu Göstergelerin Hesaplanması

Birden fazla göstergeyi hesaplamak için bir [IndicatorSet](https://www.pybroker.com/en/latest/reference/pybroker.indicator.html#pybroker.indicator.IndicatorSet) kullanılabilir. ```cmma_20``` ve ```hhv_5``` göstergeleri ```IndicatorSet```e eklenerek birlikte hesaplanabilir. Ortaya çıkan çıktı, her ikisini de içeren bir [Pandas DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) olacaktır:

In [14]:
from pybroker import IndicatorSet

indicator_set = IndicatorSet()
indicator_set.add(cmma_20, hhv_5)
indicator_set(df)

Computing indicators...


  0% (0 of 2) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--
 50% (1 of 2) |#############             | Elapsed Time: 0:00:02 ETA:   0:00:02
100% (2 of 2) |##########################| Elapsed Time: 0:00:02 Time:  0:00:02


,symbol,date,cmma_20,hhv_5
0,PG,2020-04-01,NaN,NaN
1,PG,2020-04-02,NaN,NaN
2,PG,2020-04-03,NaN,NaN
3,PG,2020-04-06,NaN,NaN
4,PG,2020-04-07,NaN,120.059998
...,...,...,...,...
500,PG,2022-03-25,1.967502,153.919998
501,PG,2022-03-28,3.288005,153.919998
502,PG,2022-03-29,4.968507,156.470001
503,PG,2022-03-30,3.790999,156.470001


## TA-Lib'i kullanma

[TA-Lib](https://github.com/TA-Lib/ta-lib-python), birçok finansal göstergeyi uygulayan, yaygın olarak kullanılan bir teknik analiz kütüphanesidir. TA-Lib'i **PyBroker** ile entegre etmek basittir. İşte bir örnek:

In [15]:
import talib

rsi_20 = pybroker.indicator('rsi_20', lambda data: talib.RSI(data.close, timeperiod=20))
rsi_20(df)

2020-04-01          NaN
2020-04-02          NaN
2020-04-03          NaN
2020-04-06          NaN
2020-04-07          NaN
                ...    
2022-03-25    49.373093
2022-03-28    51.014810
2022-03-29    53.407971
2022-03-30    51.610544
2022-03-31    49.029540
Length: 505, dtype: float64